In [0]:
%pip install --upgrade -qqqq databricks-sdk
%restart_python

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
LAKEBASE_INSTANCE_NAME = 'conversational-commerce-search-lakebase'
LAKEBASE_DB_NAME = 'order_management'


In [0]:
from databricks.sdk import WorkspaceClient

w = WorkspaceClient()
user_email = w.current_user.me().user_name
DATABRICKS_HOST = w.config.host

In [0]:
from databricks.sdk.service.database import DatabaseInstance
import time

instance = w.database.get_database_instance(name=LAKEBASE_INSTANCE_NAME)
print(instance)

DatabaseInstance(name='conversational-commerce-search-lakebase', capacity='CU_4', child_instance_refs=[], creation_time='2025-11-01T01:35:52Z', creator='dipankar.kushari@databricks.com', custom_tags=[], effective_capacity='CU_4', effective_custom_tags=[], effective_enable_pg_native_login=False, effective_enable_readable_secondaries=False, effective_node_count=2, effective_retention_window_in_days=7, effective_stopped=False, effective_usage_policy_id='4635ae18-e8d8-4528-98d3-05805c7e6308', enable_pg_native_login=None, enable_readable_secondaries=None, node_count=None, parent_instance_ref=None, pg_version='PG_VERSION_16', read_only_dns='instance-ro-42a279aa-6f5b-48df-9be3-07200d16f9a5.database.cloud.databricks.com', read_write_dns='instance-42a279aa-6f5b-48df-9be3-07200d16f9a5.database.cloud.databricks.com', retention_window_in_days=None, state=<DatabaseInstanceState.AVAILABLE: 'AVAILABLE'>, stopped=None, uid='42a279aa-6f5b-48df-9be3-07200d16f9a5', usage_policy_id=None)


In [0]:
import psycopg2
import uuid

default_database_name = 'databricks_postgres'
target_database_name = LAKEBASE_DB_NAME

cred = w.database.generate_database_credential(request_id=str(uuid.uuid4()), instance_names=[LAKEBASE_INSTANCE_NAME])

conn = psycopg2.connect(
    host = instance.read_write_dns,
    dbname = default_database_name,
    user = user_email,
    password = cred.token,
    sslmode = "require"
)

# Execute query
conn.set_isolation_level(0)
with conn.cursor() as cursor:
    cursor.execute("SELECT version()")
    version = cursor.fetchone()[0]
    print(version)
    cursor.execute(f"SELECT 1 FROM pg_database WHERE datname = '{target_database_name}'")
    if not cursor.fetchone():
        cursor.execute(f'CREATE DATABASE "{target_database_name}"')
    else:
        print('database already existed, skipping')
    cursor.close()
conn.close()

PostgreSQL 16.9 (165f042) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 11.4.0-1ubuntu1~22.04.2) 11.4.0, 64-bit
database already existed, skipping


In [0]:
app_service_principal_name="app-1ywsev conversational-commerce-search"
app_service_principal_id="19c34b6e-156c-4d90-ab88-6d264737cf86"

In [0]:
print("creating connection")
conn = psycopg2.connect(
    host = instance.read_write_dns,
    dbname = LAKEBASE_DB_NAME,
    user = user_email,
    password = cred.token,
    sslmode = "require"
)
conn.set_isolation_level(0) 
with conn.cursor() as cursor:
    print("creating table...")
    cursor.execute("CREATE EXTENSION IF NOT EXISTS databricks_auth")
    cursor.execute(f"CREATE SCHEMA IF NOT EXISTS orderdb")
    cursor.execute(f"""
        CREATE TABLE IF NOT EXISTS ORDERDB.ORDER_HEADER (
            order_id                VARCHAR(50)     PRIMARY KEY,
            order_timestamp         TIMESTAMP       NOT NULL,
            customer_name           VARCHAR(255)    NOT NULL,
            customer_email          VARCHAR(255)    NOT NULL,
            phone                   VARCHAR(20),
            address                 VARCHAR(500),
            city                    VARCHAR(100),
            state                   VARCHAR(2),
            zip_code                VARCHAR(10),
            payment_method          VARCHAR(20),
            card_last_four          VARCHAR(4),
            subtotal                DECIMAL(10,2)   NOT NULL,
            tax                     DECIMAL(10,2)   NOT NULL,
            total_price             DECIMAL(10,2)   NOT NULL,
            created_at              TIMESTAMP       NOT NULL DEFAULT CURRENT_TIMESTAMP
            );
    """)
    cursor.execute(f"""
        CREATE TABLE IF NOT EXISTS ORDERDB.ORDER_DETAILS (
            order_detail_id         VARCHAR(50)     PRIMARY KEY,
            order_id                VARCHAR(50)     NOT NULL,
            line_number             INT             NOT NULL,
            product_id              VARCHAR(50)     NOT NULL,
            name                    VARCHAR(255)    NOT NULL,
            category                VARCHAR(100)    NOT NULL,
            brand                   VARCHAR(100)    NOT NULL,
            price                   DECIMAL(10,2)   NOT NULL,
            quantity                INT             NOT NULL,
            line_total              DECIMAL(10,2)   NOT NULL,
            created_at              TIMESTAMP       NOT NULL DEFAULT CURRENT_TIMESTAMP,
            FOREIGN KEY (order_id) REFERENCES ORDERDB.ORDER_HEADER(order_id) ON DELETE CASCADE
            );
    """)
    cursor.execute(f"""GRANT ALL PRIVILEGES ON DATABASE {LAKEBASE_DB_NAME} TO PUBLIC""")
    cursor.execute(f"""GRANT ALL PRIVILEGES ON DATABASE {LAKEBASE_DB_NAME} TO \"{app_service_principal_id}\"""")
    cursor.execute(f"""GRANT ALL PRIVILEGES ON SCHEMA ORDERDB TO \"{app_service_principal_id}\"""")
    cursor.execute(f"""GRANT ALL PRIVILEGES ON ALL TABLES IN SCHEMA orderdb TO \"{app_service_principal_id}\"""")
conn.close()

creating connection
creating table...


In [0]:
# import apps

# app_name = f"vibe-coding-{username.replace('_', '-')}"[0:30]
# app = apps.get_or_create_app(wc=w, name=app_name)
# print(app_name)

In [0]:
# create service principal & secret for service principal
# import service_principals
# from databricks.sdk.service.oauth2 import ServicePrincipalSecretsProxyAPI

# sp = service_principals.get_or_create_service_principal(w, local_service_principal_name)

# secret = w.service_principal_secrets_proxy.create(service_principal_id=sp.id)

# DATABRICKS_CLIENT_ID = sp.application_id
# DATABRICKS_CLIENT_SECRET = secret.secret

In [0]:
# import groups

# group = groups.create_group_if_not_exists(wc=w, group_name=group_name)

In [0]:
# For your env:
print(f"""
      COPY/PASTE THESE INTO YOUR .env FILE ON YOUR MACHINE!
    {LAKEBASE_INSTANCE_NAME=}\n
    {LAKEBASE_DB_NAME=}\n
    {DATABRICKS_HOST=}
""")


      COPY/PASTE THESE INTO YOUR .env FILE ON YOUR MACHINE!
    LAKEBASE_INSTANCE_NAME='conversational-commerce-search-lakebase'

    LAKEBASE_DB_NAME='order_management'

    DATABRICKS_HOST='https://e2-demo-west.cloud.databricks.com'



In [0]:
# displayHTML(f'<a style="font-size: 30px" href="/settings/workspace/identity-and-access/groups/{group.id}">Click this link, and add "local-service-principal" and your App\'s service principal ({app.service_principal_client_id}) to the group!</a>')
# print(app.service_principal_client_id + " <--- your app's service principal for easy copying")
# print(sp.application_id + " <--- your local development service principal for easy copying")

In [0]:
# # THIS IS A TEST TO SEE IF YOU SUCCESSFULLY ADDED YOUR LOCAL SERVICE PRINCIPAL TO THE ABOVE GROUP PROPERLY

# from databricks.sdk import WorkspaceClient
# w2 = WorkspaceClient(
#     host = w.config.host,
#     client_id = DATABRICKS_CLIENT_ID,
#     client_secret = DATABRICKS_CLIENT_SECRET
# )

# conn = lakebase.create_lakebase_connection(wc=w2, instance=instance, db_name=LAKEBASE_DB_NAME, user=group_name)

# with conn.cursor() as cursor:
#     cursor.execute(f"""
#         SELECT * from public.{username}_lists;
#     """)
#     res = cursor.fetchall()
#     print(res)
# conn.close()